In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from copy import deepcopy

from autowoe import ReportDeco, AutoWoE

### Чтение и подготовка обучающей выборки

In [ ]:
train = pd.read_csv("./example_data/train_demo.csv",
                    low_memory=False,
                    index_col="line_id",
                    parse_dates = ["datetime_" + str(i) for i in range(2)],)

train = train.iloc[:, 50:100]

num_col = list(filter(lambda x: "numb" in x, train.columns))
num_feature_type = {x: "real" for x in num_col}

date_col = filter(lambda x: "datetime" in x, train.columns)
for col in date_col:
    train[col + "_year"] = train[col].map(lambda x: x.year)
    train[col + "_weekday"] = train[col].map(lambda x: x.weekday())
    train[col + "_month"] = train[col].map(lambda x: x.month)

### Чтение и подготовка тестовой выборки

In [ ]:
test = pd.read_csv("./example_data/test_demo.csv",
                   index_col="line_id", 
                   parse_dates = ["datetime_" + str(i) for i in range(2)])

date_col = filter(lambda x: "datetime" in x, test.columns)
for col in date_col:
    test[col + "_year"] = test[col].map(lambda x: x.year)
    test[col + "_weekday"] = test[col].map(lambda x: x.weekday())
    test[col + "_month"] = test[col].map(lambda x: x.month)
    
test_target = pd.read_csv("./example_data/test-target_demo.csv")["target"]
test["target"] = test_target.values

### Параметры модели

Для обучения модели рекомендуется указать тип признаков для обучения.
Поэтому создается словарь features_type с ключами: 


"real" -- вещественный признак,

"cat" --  категориальный.

Для признаков, которые не размечены, типы будут определены автоматом. Такой вариант будет работать, но качество порядочно просядет

#### features_type

In [ ]:
cat_col = list(filter(lambda x: "str" in x, train.columns))
cat_feature_type = {x: "cat" for x in cat_col}

year_col = list(filter(lambda x: "_year" in x, train.columns))
year_feature_type = {x: "cat" for x in year_col}

weekday_col = list(filter(lambda x: "_weekday" in x, train.columns))
weekday_feature_type = {x: "cat" for x in weekday_col}

month_col = list(filter(lambda x: "_month" in x, train.columns))
month_feature_type = {x: "cat" for x in month_col}

In [ ]:
features = cat_col + year_col + weekday_col + month_col + num_col

#### Feature level constrains

In [ ]:
features_type = dict(**num_feature_type,
                     **cat_feature_type,
                     **year_feature_type,
                     **weekday_feature_type,
                     **month_feature_type)

- `features_monotone_constraints` - также можно указать зависимость целевой переменной от признака. Если заранее известно, что при возрастании признака feature_1, то эту информацию можно учесть в модели, добавив в словарь пару {feature_1: "1"}. Если же зависимость признака от целевой переменной обратная, то можно указать {feature_1: "-1"} Если про зависимость ничего неизвестно, но хочется, чтобы она была монотонная, можно указать 'auto'. Можно указать  {feature_1: "0"}, в случае, если установлено общее ограничение на монотонность, чтобы не распространять его на эту фичу. Если специальных условий нет, то можно не собирать этот дикт


Рекомендуемое использование:

1) В случае, если задано общее условие на монотонность, то можно собрать дикт {feature_1: "0", feature_2: "0"}, чтобы игнорировать это ограничение для признаков feature_1, feature_2

2) В случае, если не задано общее условие на монотонность, то можно собрать дикт {feature_1: "auto", feature_2: "auto"}, чтобы установить это ограничение для признаков feature_1, feature_2

In [ ]:
features_monotone_constraints = {'number_74': 'auto',  'number_83': 'auto'} 

- `max_bin_count`  - через словарь max_bin_count можно задать число бинов для WoE кодирования, если для какого-то признака оно отлично от общего. 

In [ ]:
max_bin_count = {'number_47': 3, 'number_51': 2}

####  Рекомендация
В общем случае, в первый момент построения модели лучше не указывать специальных ограничений в features_monotone_constraints и max_bin_count. Если в результате анализа полученной модели разбиение оказалось неинтерпретируемым или нестабильным по отдельным признакам, но в целом по модели ок, то ограничить сложность разбиения отдельных призаков имеет смысл. Если разбивка большинства признаков в модели оказалась неудовлетворительная, то рекомендуется в первую очередь настраивать глобальные ограничения (см параметры модели max_bin_count, monotonic, min_bin_size и др ниже)

####  Общие параметры модели

- `interpreted_model` - требуется ли интерпретируемость модели (условие на знак в коэффициентах логистической регрессии)

- `monotonic` - Глобальное условие на монотонность. Если указано True, то для всех признаков по умолчанию будут строится только монотонные разбиения. Указать специальные условия для отдельных признаков можно используя features_monotone_constraints аргумент метода .fit

- `max_bin_count` - Глобальное ограничение на число бинов. Указать специальные условия для отдельных признаков можно используя max_bin_count аргумент метода .fit

- `select_type`  - способ ПРЕДВАРИТЕЛЬНОГО!!! (ЭТО ВАЖНО) отбора признаков. Если указать None, то будут отобраны признаки, у которых importance больше imp_th. Если указвать, например 50, то после предварительного отобра останется только 50 признаков самых важных признаков. Крайне не рекомендуется сильно ограничивать

- `pearson_th` - пороговое значение для корреляции Пирсона. Используется на финальной стадии отбора признаков.
Если корреляция вух признаков по модулю больше pearson_th, то будет выброшен тот, у которого 
информативность меньше

- `auc_th` - пороговое значение для одномерной оценки качества признака

- `vif_th` - пороговое значение для VIF признака

- `imp_th` - порог по которому будет произведен отбор признаков, если указать select_type=None (см. ниже).

- `th_const` порог по которому признак будет считаться константным. Все константные признаки в модели не учитываются. Если число валидных значений больше трешхолда, то колонка не константная (int). В случае указания float, трешхолд будет определяться как размер_выборки * th_const

- `force_single_split` - иногда в силу ограничений на min_bin_size невозможно построить ни одной группировки на переменную. force_single_split=True заставит в этом случае построить единственно возмоджный сплит, в случае если при этом выделяется группа размера более чем th_const. False будет выкидывать этот признак


- `th_nan` - порог по которому будет выделена отдельная категория для пропусков в данных.
Если число пропусков меньше чем th_nan, то WoE значения для пропусков берется равным нулю.
В противном случае пропущенные значения будут выделены в отдельную группу и для них отдельно
будет рассчитано WoE значение.
Так же влияет на редкие категории (менее th_cat). Если суммарно таких категорий будет менее th_nan, то обработка будет производиться по принципу отпределенному в `cat_merge_to`, иначе оценено по группе

- `th_cat` - порог, по которой немногочисленные категории в категориальных признаках будут объединятся в отдельную группу


- `woe_diff_th` - Возмодность смеджить наны и редкие категории с каким-то бином, если разница в вое менее woe_diff_th


- `min_bin_size` - минимальный размер бина при группировке. Возможно int как число наблюдений и float как доля от выбрки

- `min_bin_mults` - в ходе построения бинов будут протестированы возможные значения min_bin_size, 
min_bin_size * min_bin_mults[0], min_bin_size * min_bin_mults[1] ... . Ждем float > 1. Дефолт - (2, 4), в принципе можно не трогать

- `min_gains_to_split` - возможные значения регуляризатора, которые будут протестированы в ходе построения биннинга


- `auc_tol` - Чувствительность к AUC. Считаем, что можем пожертвовать auc_tol качества от максимального, чтобы сделать модель проще


- `cat_alpha` - Регуляризатор для кодировщика категорий



- `cat_merge_to` - группа для редких (менее th_cat) категорий либо новых на тесте
         "to_nan" -- в группу nan, 
         "to_woe_0" -- отдельная группа с WoE = 0,
         "to_maxfreq" - в самую большую группу,
         "to_maxp" - в группу с наибольшей вероятностью события,
         "to_minp" - в группу с наименьшей вероятностью события
         
- `nan_merge_to` - группа для НаНов
         "to_woe_0" -- отдельная группа с WoE = 0,
         "to_maxfreq" - в самую большую группу,
         "to_maxp" - в группу с наибольшей вероятностью события,
         "to_minp" - в группу с наименьшей вероятностью события  
         
         
- `oof_woe` - если указать oof_woe=True, то WoE кодирование будет происходить по кросс-валидации. Если же False, то сразу на всей обучающей выборке.

- `n_folds` - количество фолдов для внутренней кроссвалидации


- `n_jobs` - число процессов, которое будет использовать модель 

- `l1_grid_size` - в данной модели на одном из шагов используется отбор признаков LASSO. l1_base_step -- размер сетки для перебора C

- `l1_exp_scale` - шкала сетки для L1 отбора. 4 соответствует макс значению C порядка 3-4. Увеличивать, если необходимо сделать менее регуляризованную модель

- `imp_type` - способ определения значимости признаков -- features importance ("feature_imp" - в общем случае более сложная модель) или permutation importance ("perm_imp" - в общем случае более простая модель)

- `regularized_refit` - после отбора признаков полученная модель пересчитывается на всех данных. Стоит ли включать L1 при этом. Если нет, то в интерпретируемом режиме модель будет итеративно переобучаться, пока все веса не станут отрицательны. Если да - то аналогичное будет получаться закручиванием L1. Может быть полезно ставить False если нужна стат модель, те p-value на оценки

- `p_val` - допустимый уровень p_value на оценки модели при условии обучении стат модели (regularized_refit=False)

In [ ]:
auto_woe = AutoWoE(interpreted_model=True,
                     monotonic=False,
                     max_bin_count=5,
                     select_type=None,
                     pearson_th=0.9,
                     auc_th=.505,
                     vif_th=10.,
                     imp_th=0,
                     th_const=32,
                     force_single_split=True,
                     th_nan=0.01,
                     th_cat=0.005,
                     woe_diff_th=0.01,
                     min_bin_size=0.01,
                     min_bin_mults=(2, 4),
                     min_gains_to_split=(0.0, 0.5, 1.0),
                     auc_tol=1e-4,
                     cat_alpha=100,
                     cat_merge_to="to_woe_0",
                     nan_merge_to="to_woe_0",
                     oof_woe=True,
                     n_folds=6,
                     n_jobs=4,
                     l1_grid_size=20,
                     l1_exp_scale=6,
                     imp_type="feature_imp",
                     regularized_refit=False,
                     p_val=0.05,
                     debug=False,
                     verbose=0
        )

auto_woe = ReportDeco(auto_woe)

- `train` обучающая выборка

- `target_name` - название целевой переменной

- `features_type` - см выше описание дикта features_type. Возможно указание None для автозаполнения, но не рекомендуется

- `group_kf` -  название колонки-группы для GroupKFold https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html

- `max_bin_count` - см выше описание дикта max_bin_count. Можно ничего не передавать, если специальных условий не предусмотрено. Общее для всех условние задано в __init__

- `features_monotone_constraints` - см выше описание дикта features_monotone_constraints. Можно ничего не передавать, если специальных условий не предусмотрено. Общее для всех условние задано в __init__

- `validation` - возможность использовать валидацию в построении/отборе признаков. Можно не передавать. На текущий момент используется для 1) отбора признаков по p-value при построении стат модели


In [ ]:
auto_woe.fit(train[features + ['target']], 
             target_name="target",
             features_type=features_type,
             group_kf=None,
             max_bin_count=max_bin_count,
             features_monotone_constraints=features_monotone_constraints,
             validation=test
            )

In [ ]:
pred = auto_woe.predict_proba(test)
roc_auc_score(test['target'], pred)

In [ ]:
pred = auto_woe.predict_proba(test[['number_72']], report=False)
roc_auc_score(test['target'], pred)


In [ ]:
print(auto_woe.get_sql_inference_query('table'))

### Полезные методы модели

- `private_features_type` - типизация признаков
- `get_woe` - рабиение на бины и WoE значения в них
- `get_split` - границы разбиения. Особо полезен для категориальных признаков


##### Замечание: 
ReportDeco - обертка для построения отчета. Она не обязательна для обучения и применения модели, но обязательна для построения отчета (см последнюю ячейку).
Для доступа к атрибутам самой модели необходимо обратится к атрибуту auto_woe.model декоратора
Все атрибуты объекта-модели так же доступны через объект-отчета.
Однако в пикл отчета будет весить существенно больше, так что для сохранения модели на инференс стоит сохранять только auto_woe.model


### Формирование отчета

In [ ]:
report_params = {"automl_date_column": "report_month", # колонка с датой в формате params['datetimeFormat']
                 "output_path": "./AUTOWOE_REPORT_1", # папка, куда сгенерится отчет и сложатся нужные файлы
                 "report_name": "___НАЗВАНИЕ ОТЧЕТА___",
                 "report_version_id": 1,
                 "city": "Воронеж",
                 "model_aim": "___ЦЕЛЬ ПОСТРОЕНИЯ МОДЕЛИ___",
                 "model_name": "___НАЗВАНИЕ МОДЕЛИ___",
                 "zakazchik": "___ЗАКАЗЧИК___",
                 "high_level_department": "___ПОДРАЗДЕЛЕНИЕ___",
                 "ds_name": "___РАЗРАБОТЧИК МОДЕЛИ___",
                 "target_descr": "___ОПИСАНИЕ ЦЕЛЕВОГО СОБЫТИЯ___",
                 "non_target_descr": "___ОПИСАНИЕ НЕЦЕЛЕВОГО СОБЫТИЯ___"}

auto_woe.generate_report(report_params)